In [1]:
import shutil
from decimal import Decimal
from pathlib import Path

import pandas as pd

from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model import BarType, InstrumentId, instruments, BarSpecification, Venue
from nautilus_trader.model.enums import BarAggregation, PriceType, AssetClass, OmsType, AccountType, AggregationSource
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import BarDataWrangler
from nautilus_trader.test_kit.providers import CSVBarDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider
from nautilus_trader.model import Symbol
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Bar

In [3]:
from nautilus_trader.core.datetime import unix_nanos_to_dt

catalog = ParquetDataCatalog("./catalog")
bar = BarType(
    InstrumentId.from_str("VOO.NASDAQ"),
    BarSpecification(30, BarAggregation.MINUTE, PriceType.LAST),
    AggregationSource.INTERNAL
)

bars = catalog.bars(bar)

with open("test.txt", "w+") as f:
    for b in bars:
        f.write(f"{unix_nanos_to_dt(b.ts_event)} | {b.close}, {b.volume}\n")

In [ ]:
bar